# Prep stuff

In [1]:
drive_path = 'c:/'
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
from scipy.stats import anderson_ksamp
from scipy.stats import kruskal
from scipy.stats import variation
%matplotlib
import seaborn as sns

Using matplotlib backend: Qt4Agg


In [63]:
#composite.head()

In [64]:
#sns.boxplot(x="Odor", y="value", hue="Group", data=comp_tmp);

In [65]:
#sns.violinplot(x="Odor", y="value", hue="Group", data=comp_tmp);

In [66]:
#sns.barplot(x="Odor", y="value", hue="Group", data=comp_tmp);

# Import/organize data

In [3]:
comp=pd.read_csv('C:\Users\Annie\Documents\Data\Ca_Imaging\Analysis\Odor_Panel\Composite_MaxDF_NoP.csv')
del comp['Mouse']
comp_sorted=comp.reindex_axis(comp.mean().sort_values().index, axis=1)
comp_labels=pd.DataFrame(comp.Group)
tmp=[comp_labels,comp_sorted]
composite_full=pd.concat(tmp,axis=1)
composite_full.head()
cfull=pd.melt(composite_full,"Group",var_name="Odor")

# Rank the odors

In [3]:
composite_full.head()

,Group,THA,MS 0.01,BLANK,AP,MS 0.1,MS 0.05,IAA 0.05,IAA 0.01,PA,IAA 0.1,Hexanone,Hexanal 0.1,Hexanal 0.01,Hexanal 0.05,EB
0,Control,0.212200,0.127301,NaN,0.139029,0.096683,0.099481,0.098134,0.265576,0.165395,0.216148,0.345297,0.131523,0.157010,0.131461,0.193486
1,Control,0.066122,0.074838,NaN,0.127978,0.052906,0.119782,0.108896,0.168569,0.101373,0.171575,0.268874,0.146749,0.140497,0.136131,0.201126
2,Control,0.022259,0.094999,NaN,0.065581,0.027137,0.029730,0.028554,0.146171,0.055140,0.159719,0.198244,0.101923,0.127977,0.116245,0.156971
3,Control,0.145203,0.107146,NaN,0.057500,0.152532,0.158519,0.179303,0.302537,0.125637,0.259681,0.350995,0.279234,0.190135,0.233960,0.274613
4,Control,0.155862,0.147167,NaN,0.153746,0.125200,0.159181,0.230083,0.375661,0.192976,0.325386,0.438699,0.357176,0.299774,0.339993,0.285755


In [3]:
x=composite_full.rank(axis=1,numeric_only=float,na_option='keep',ascending=False)
comp_labels=pd.DataFrame(comp.Group)
tmp=[comp_labels,x]
comp_ranks=pd.concat(tmp,axis=1)
comp_ranks.head()

,Group,THA,MS 0.01,BLANK,AP,MS 0.1,MS 0.05,IAA 0.05,IAA 0.01,PA,IAA 0.1,Hexanone,Hexanal 0.1,Hexanal 0.01,Hexanal 0.05,EB
0,Control,4.0,11.0,NaN,8.0,14.0,12.0,13.0,2.0,6.0,3.0,1.0,9.0,7.0,10.0,5.0
1,Control,13.0,12.0,NaN,8.0,14.0,9.0,10.0,4.0,11.0,3.0,1.0,5.0,6.0,7.0,2.0
2,Control,14.0,8.0,NaN,9.0,13.0,11.0,12.0,4.0,10.0,2.0,1.0,7.0,5.0,6.0,3.0
3,Control,11.0,13.0,NaN,14.0,10.0,9.0,8.0,2.0,12.0,5.0,1.0,3.0,7.0,6.0,4.0
4,Control,11.0,13.0,NaN,12.0,14.0,10.0,8.0,2.0,9.0,5.0,1.0,3.0,6.0,4.0,7.0


In [4]:
ctrl_cr=comp_ranks[comp_ranks['Group']=='Control']
mint_cr=comp_ranks[comp_ranks['Group']=='Mint']
hex_cr=comp_ranks[comp_ranks['Group']=='Hexanal']

In [10]:
pd.DataFrame({'Group':['Control','Mint','Hexanal']})

,Group
0,Control
1,Mint
2,Hexanal


In [20]:
ctrlmr=pd.DataFrame(comp_ranks[comp_ranks['Group']=='Control'].mean(axis=0)).T
mintmr=pd.DataFrame(comp_ranks[comp_ranks['Group']=='Mint'].mean(axis=0)).T
hexmr=pd.DataFrame(comp_ranks[comp_ranks['Group']=='Hexanal'].mean(axis=0)).T
mintmr

,THA,MS 0.01,BLANK,AP,MS 0.1,MS 0.05,IAA 0.05,IAA 0.01,PA,IAA 0.1,Hexanone,Hexanal 0.1,Hexanal 0.01,Hexanal 0.05,EB
0,9.98694,11.625,8.68254,10.794776,9.296642,9.910448,8.175373,8.891791,8.975746,5.929104,5.076493,3.953358,5.01306,4.009328,4.847015


In [41]:
ranks=pd.concat([ctrlmr,mintmr,hexmr])
grouplabels=pd.DataFrame({'Group':['Control','Mint','Hexanal']})
ranks=ranks.reset_index(drop=True)
# tmp=[grouplabels,ranks]
finaldf=pd.concat([grouplabels,ranks],axis=1)
finaldf

,Group,THA,MS 0.01,BLANK,AP,MS 0.1,MS 0.05,IAA 0.05,IAA 0.01,PA,IAA 0.1,Hexanone,Hexanal 0.1,Hexanal 0.01,Hexanal 0.05,EB
0,Control,10.535714,10.415414,11.667647,10.372180,9.530075,9.836466,9.105263,6.765038,10.806391,5.052632,3.759398,5.731203,5.986842,4.118421,5.114662
1,Mint,9.986940,11.625000,8.682540,10.794776,9.296642,9.910448,8.175373,8.891791,8.975746,5.929104,5.076493,3.953358,5.013060,4.009328,4.847015
2,Hexanal,11.222222,12.147696,9.899425,9.535230,11.214092,10.284553,10.021680,7.728997,8.398374,4.802168,5.326558,6.643631,4.586721,3.162602,4.735772


In [44]:
finalmelt=pd.melt(finaldf,"Group",var_name="Odor")

In [42]:
kruskal(ctrl_cr.EB,mint_cr.EB,hex_cr.EB)

KruskalResult(statistic=11.08517822319291, pvalue=0.0039163737715180091)

In [46]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk", font_scale=1.8);
plt.figure(figsize=(55, 20));
sns.pointplot(x="Odor", y="value", hue="Group", data=finalmelt,
              palette={"Control": "r", "Mint": "g", 'Hexanal': 'b'});
sns.despine()
plt.ylabel('Rank', fontsize=48);
plt.title('Rank of odor response', fontsize=55);
plt.xlabel('Odor', fontsize=48);
plt.legend(loc=2, prop={'size': 48});

# Mean, Median, COV

Calculate Mean

In [109]:
#Calculate means for each odor
Cctrl=composite_full[composite_full['Group']=='Control']
Cmean=Cctrl.mean()
M=composite_full[composite_full['Group']=='Mint']
Mmean=M.mean()
H=composite_full[composite_full['Group']=='Hexanal']
Hmean=H.mean()

CtrlMDFT=pd.DataFrame(Cmean).transpose()
MMDFT=pd.DataFrame(Mmean).transpose()
HMDFT=pd.DataFrame(Hmean).transpose()

#add group labels back
gnc = pd.DataFrame({'Group':['Control']})
gnm=pd.DataFrame({'Group':['Mint']})
gnh=pd.DataFrame({'Group':['Hexanal']})

Ctmp=[gnc,CtrlMDFT]
Mtmp=[gnm,MMDFT]
Htmp=[gnh,HMDFT]

CtrlMDF=pd.concat(Ctmp,axis=1)
MMDF=pd.concat(Mtmp,axis=1)
HMDF=pd.concat(Htmp,axis=1)

final=[CtrlMDF,MMDF,HMDF]
finalmean=pd.concat(final)
finalmeandf=pd.melt(finalmean,"Group",var_name="Odor")

In [157]:
#plot means
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.8);
plt.figure(figsize=(55,20));
sns.pointplot(x="Odor", y="value", hue="Group", data=finalmeandf,palette={"Control": "r", "Mint": "g",'Hexanal':'b'});
sns.despine()
plt.ylabel('DF/F', fontsize=48);
plt.title('Mean Peak DF/F',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

Calculate Medians

In [8]:
#Calculate medians for each odor
Cctrl=composite_full[composite_full['Group']=='Control']
Cmedian=Cctrl.median()
M=composite_full[composite_full['Group']=='Mint']
Mmedian=M.median()
H=composite_full[composite_full['Group']=='Hexanal']
Hmedian=H.median()

CtrlMedT=pd.DataFrame(Cmedian).transpose()
MMedT=pd.DataFrame(Mmedian).transpose()
HMedT=pd.DataFrame(Hmedian).transpose()

#add group labels back
gnc = pd.DataFrame({'Group':['Control']})
gnm=pd.DataFrame({'Group':['Mint']})
gnh=pd.DataFrame({'Group':['Hexanal']})

Ctmp=[gnc,CtrlMedT]
Mtmp=[gnm,MMedT]
Htmp=[gnh,HMedT]

CtrlMed=pd.concat(Ctmp,axis=1)
MMed=pd.concat(Mtmp,axis=1)
HMed=pd.concat(Htmp,axis=1)

finalmed=[CtrlMed,MMed,HMed]
finalmedian=pd.concat(finalmed)
finalmediandf=pd.melt(finalmedian,"Group",var_name="Odor")

In [9]:
#Plot medians
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.8);
plt.figure(figsize=(55,20));
sns.pointplot(x="Odor", y="value", hue="Group", data=finalmediandf,palette={"Control": "r", "Mint": "g",'Hexanal':'b'});
sns.despine()
plt.ylabel('DF/F', fontsize=48);
plt.title('Median Peak DF/F',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

Calculate and Plot Variance

In [10]:
# Calculate variance
CC=composite_full[composite_full['Group']=='Control']
MM=composite_full[composite_full['Group']=='Mint']
HH=composite_full[composite_full['Group']=='Hexanal']
del CC['Group']
del MM['Group']
del HH['Group']

CvarT=pd.DataFrame(variation(CC)).transpose()
MvarT=pd.DataFrame(variation(MM)).transpose()
HvarT=pd.DataFrame(variation(HH)).transpose()

#add group labels back
gnc = pd.DataFrame({'Group':['Control']})
gnm=pd.DataFrame({'Group':['Mint']})
gnh=pd.DataFrame({'Group':['Hexanal']})

Ctmp=[gnc,CvarT]
Mtmp=[gnm,MvarT]
Htmp=[gnh,HvarT]

CV=pd.concat(Ctmp,axis=1)
MV=pd.concat(Mtmp,axis=1)
HV=pd.concat(Htmp,axis=1)

gn=['Group']
gname = np.array(gn)
odornames=CC.columns.values
columnnames=np.append(gname,odornames)
CV.columns=columnnames
MV.columns=columnnames
HV.columns=columnnames

#Fix BLANK NaNs
CCNB=pd.DataFrame(CC['BLANK'])
CCNB=CCNB.dropna()
CCNBvar=variation(CCNB)
CCNBvars=np.asscalar(CCNBvar)
CV=CV.replace(CV.BLANK[0],CCNBvars)

MMNB=pd.DataFrame(MM['BLANK'])
MMNB=MMNB.dropna()
MMNBvar=variation(MMNB)
MMNBvars=np.asscalar(MMNBvar)
MV=MV.replace(MV.BLANK[0],MMNBvars)

HHNB=pd.DataFrame(HH['BLANK'])
HHNB=HHNB.dropna()
HHNBvar=variation(HHNB)
HHNBvars=np.asscalar(HHNBvar)
HV=HV.replace(HV.BLANK[0],HHNBvars)

complete=[CV,HV,MV]
finalvariance=pd.concat(complete)
finalvariancedf=pd.melt(finalvariance,"Group",var_name="Odor")

In [11]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.8);
plt.figure(figsize=(55,20));
sns.pointplot(x="Odor", y="value", hue="Group", data=finalvariancedf,palette={"Control": "r", "Mint": "g",'Hexanal':'b'});
sns.despine()
plt.ylabel('COV', fontsize=48);
plt.title('COV Peak DF/F',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

Calculate and plot variance for each odor concentration

In [12]:
MSconcvar=finalvariance[['Group','MS 0.01','MS 0.05','MS 0.1']]
Hexconcvar=finalvariance[['Group','Hexanal 0.01','Hexanal 0.05','Hexanal 0.1']]
IAAconcvar=finalvariance[['Group','IAA 0.01','IAA 0.05','IAA 0.1']]

MSconcvardf=pd.melt(MSconcvar,"Group",var_name="Odor")
Hexconcvardf=pd.melt(Hexconcvar,'Group', var_name='Odor')
IAAconcvardf=pd.melt(IAAconcvar,'Group',var_name='Odor')

In [13]:
#Plot COV MS Concentration
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.8);
plt.figure(figsize=(55,20));
sns.pointplot(x="Odor", y="value", hue="Group", data=MSconcvardf,palette={"Control": "r", "Mint": "g",'Hexanal':'b'});
sns.despine()
plt.ylabel('COV', fontsize=48);
plt.title('COV Peak DF/F - Methyl Salicylate',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

In [211]:
# Plot COV Hexanal Concentration
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.8);
plt.figure(figsize=(55,20));
sns.pointplot(x="Odor", y="value", hue="Group", data=Hexconcvardf,palette={"Control": "r", "Mint": "g",'Hexanal':'b'});
sns.despine()
plt.ylabel('COV', fontsize=48);
plt.title('COV Peak DF/F - Hexanal',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

In [212]:
# Plot COV IAA Concentration
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.8);
plt.figure(figsize=(55,20));
sns.pointplot(x="Odor", y="value", hue="Group", data=IAAconcvardf,palette={"Control": "r", "Mint": "g",'Hexanal':'b'});
sns.despine()
plt.ylabel('COV', fontsize=48);
plt.title('COV Peak DF/F - Isoamyl acetate',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

# Correlate Baseline with DF/F

In [3]:
#Get Baselines
bl=pd.read_csv('C:\Users\Annie\Documents\Data\Ca_Imaging\Analysis\Odor_Panel\Composite_Baseline_NoP.csv')
del bl['Mouse']
bl_sorted=bl.reindex_axis(bl.mean().sort_values().index, axis=1)
bl_labels=pd.DataFrame(bl.Group)
tmp=[bl_labels,bl_sorted]
bdf=pd.concat(tmp,axis=1)
bdfull=pd.melt(bdf,"Group",var_name="Odor")
bdfull=bdfull.dropna()

In [5]:
composite_cn=composite_full.columns.tolist()
bdf=bdf[composite_cn]
bdf.head()

,Group,THA,MS 0.01,BLANK,AP,MS 0.1,MS 0.05,IAA 0.05,IAA 0.01,PA,IAA 0.1,Hexanone,Hexanal 0.1,Hexanal 0.01,Hexanal 0.05,EB
0,Control,2191.580000,2156.251500,NaN,2604.431208,2342.740333,2438.371125,2430.727208,1916.923958,2174.460083,2295.764583,2202.500042,2446.136083,2515.360375,2457.603625,2406.386167
1,Control,1733.676000,1897.166667,NaN,1805.344875,1946.681708,1828.752875,1864.458375,1685.373583,1748.060375,1850.529417,1865.847750,1903.826250,1841.318875,1795.494250,1929.843458
2,Control,1567.613708,1688.818208,NaN,1678.700042,1694.186292,1619.368000,1643.305542,1556.993000,1536.563083,1662.997500,1504.910333,1681.573792,1648.710833,1688.000667,1663.836417
3,Control,829.527875,888.688458,NaN,865.015708,856.108625,861.587750,857.491250,835.064458,822.373417,847.159000,853.859458,857.543458,871.575583,853.299500,843.551167
4,Control,886.599167,1019.396583,NaN,924.648000,929.562417,916.596208,916.315333,955.118500,895.472083,914.607042,948.426042,899.221875,918.836292,914.384250,942.410958


In [56]:
#Turn dataframe into a single column
bdfmelt=pd.melt(bdf,"Group",var_name='Odor')
cfmelt=pd.melt(composite_full,'Group',var_name='Odor')

In [86]:
#Make a dataframe so you can plot the correlation
plotcorr=pd.concat([cfmelt,bdfmelt['value']],axis=1)
plotcorr.tail()

,Group,Odor,value,value
13540,Mint,EB,0.479948,810.413917
13541,Mint,EB,0.367013,719.335792
13542,Mint,EB,0.238822,880.315292
13543,Mint,EB,0.259046,1241.160417
13544,Mint,EB,0.313997,1080.826792


In [99]:
#Rename these columns to what they're supposed to be
plotcorr.columns=['Group','Odor','DFF','Baseline']
plotcorr.head()

,Group,Odor,DFF,Baseline
0,Control,THA,0.212200,2191.580000
1,Control,THA,0.066122,1733.676000
2,Control,THA,0.022259,1567.613708
3,Control,THA,0.145203,829.527875
4,Control,THA,0.155862,886.599167


In [108]:
#Plot Baseline vs. DFF as a scatterplot
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.4);
plotcorr.plot(x='DFF',y='Baseline',kind='scatter')
plt.ylabel('Baseline Intensity');
plt.title('Peak DF/F vs. Baseline');
plt.xlabel('Peak DF/F');

In [101]:
#Calculate the spearman coefficient between DFF and Baseline
#rho is the coefficient, pval is the pvalue
from scipy.stats import spearmanr
rho,pval=spearmanr(plotcorr['DFF'],plotcorr['Baseline'])
rho

-0.30211262511970655

In [42]:
#Corr based on treatment groups
CC=composite_full[composite_full['Group']=='Control']
MM=composite_full[composite_full['Group']=='Mint']
HH=composite_full[composite_full['Group']=='Hexanal']

CB=bdf[bdf['Group']=='Control']
MB=bdf[bdf['Group']=='Mint']
HB=bdf[bdf['Group']=='Hexanal']

CCcorr=CC.corr()
MMcorr=MM.corr()
HHcorr=HH.corr()
CBcorr=CB.corr()
MBcorr=MB.corr()
HBcorr=HB.corr()

Ctrlcorr=CC.corrwith(CB)
Mintcorr=MM.corrwith(MB)
Hexcorr=HH.corrwith(HB)

CtrlCdf=pd.DataFrame(Ctrlcorr)
CtrlCdf=CtrlCdf.transpose()
MintCdf=pd.DataFrame(Mintcorr)
MintCdf=MintCdf.transpose()
HexCdf=pd.DataFrame(Hexcorr)
HexCdf=HexCdf.transpose()
tmp=[CtrlCdf,MintCdf,HexCdf]

CompleteCorrDF=pd.concat(tmp)

In [ ]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=1.8);
plt.figure(figsize=(55,20));
sns.pointplot();
sns.despine()
plt.ylabel('COV', fontsize=48);
plt.title('COV Peak DF/F - Methyl Salicylate',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

# Composite Graphs

In [317]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=2.2);
plt.figure(figsize=(45,20));
ax=sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=cfull);
sns.despine()
plt.ylabel('Peak DF/F', fontsize=48);
plt.title('Peak DF/F',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

In [14]:
# sns.set_context("talk",font_scale=1.9);
# sns.set(style="white", palette="muted", color_codes=True);
# plt.figure(figsize=(8, 6));
# violinplot=sns.violinplot(x="Odor", y="value", hue="Group", data=cfull);
# violinplot.set(ylabel='Peak DF/F');

In [318]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=2.2);
plt.figure(figsize=(45,20));
ax=sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=cfull);
sns.despine()
plt.ylabel('Peak DF/F', fontsize=48);
plt.title('Peak DF/F',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':48});

# Concentration based graphs

# MS

In [5]:
MS_full=composite_full[['Group','MS 0.01','MS 0.05','MS 0.1']]
MSdf=pd.melt(MS_full,"Group",var_name="Odor")
MSdf.head()

,Group,Odor,value
0,Control,MS 0.01,0.127301
1,Control,MS 0.01,0.074838
2,Control,MS 0.01,0.094999
3,Control,MS 0.01,0.107146
4,Control,MS 0.01,0.147167


In [325]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=4.3);
plt.figure(figsize=(30, 20));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=MSdf);
sns.despine()
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [326]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=4.3);
plt.figure(figsize=(30, 20));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=MSdf);
sns.despine()
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

# Histograms (MS)

In [4]:
MSctrl=MS_full[MS_full['Group'] == 'Control']
MSMS=MS_full[MS_full['Group'] == 'Mint']
MShex=MS_full[MS_full['Group'] == 'Hexanal']
MSctrl.tail()

NameError: name 'MS_full' is not defined

In [366]:
# Control, MS Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30, 20), sharex=True)
# f.suptitle("Control, MS Concentration", fontsize=40)
sns.despine(left=True)

#data
d = MSctrl['MS 0.01']
e = MSctrl['MS 0.05']
f = MSctrl['MS 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="MS 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="MS 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="MS 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [368]:
# Hexanal, MS Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Hexanal, MS Concentration", fontsize=44)
sns.despine(left=True)

#data
d = MShex['MS 0.01']
e = MShex['MS 0.05']
f = MShex['MS 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="MS 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="MS 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="MS 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [369]:
# Mint, MS Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Mint, MS Concentration", fontsize=44)
sns.despine(left=True)

#data
d = MSMS['MS 0.01']
e = MSMS['MS 0.05']
f = MSMS['MS 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="MS 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="MS 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="MS 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

# Hexanal

In [23]:
H_full=composite_full[['Group','Hexanal 0.01','Hexanal 0.05','Hexanal 0.1']]
Hdf=pd.melt(H_full,"Group",var_name="Odor")

In [370]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=4.3);
plt.figure(figsize=(30,20));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=Hdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [328]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=4.3);
plt.figure(figsize=(30,20));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=Hdf);
sns.despine()
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

# Histograms (Hexanal)

In [178]:
Hctrl=H_full[H_full['Group'] == 'Control']
HMS=H_full[H_full['Group'] == 'Mint']
HH=H_full[H_full['Group'] == 'Hexanal']
HH.head()

,Group,Hexanal 0.01,Hexanal 0.05,Hexanal 0.1
266,Hexanal,0.346753,0.300836,0.290083
267,Hexanal,0.269426,0.349465,0.236634
268,Hexanal,0.359969,0.422324,0.416844
269,Hexanal,0.136656,0.135468,0.124364
270,Hexanal,0.188876,0.200218,0.136488


In [373]:
# Control, Hexanal Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Control, Hexanal Concentration", fontsize=44)
sns.despine(left=True)

#data
d = Hctrl['Hexanal 0.01']
e = Hctrl['Hexanal 0.05']
f = Hctrl['Hexanal 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Hexanal 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Hexanal 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [372]:
# Hexanal, Hexanal Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Hexanal, Hexanal Concentration", fontsize=44)
sns.despine(left=True)

#data
d = HH['Hexanal 0.01']
e = HH['Hexanal 0.05']
f = HH['Hexanal 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Hexanal 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Hexanal 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [374]:
# Mint, Hexanal Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Mint, Hexanal Concentration", fontsize=44)
sns.despine(left=True)

#data
d = HMS['Hexanal 0.01']
e = HMS['Hexanal 0.05']
f = HMS['Hexanal 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Hexanal 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Hexanal 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

# IAA

In [35]:
IAA_full=composite_full[['Group','IAA 0.01','IAA 0.05','IAA 0.1']]
IAAdf=pd.melt(MS_full,"Group",var_name="Odor")

In [329]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=4.3);
plt.figure(figsize=(30,20));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAAdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [331]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=4.3);
plt.figure(figsize=(30,20));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAAdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

# Histograms (IAA)

In [182]:
Ictrl=IAA_full[IAA_full['Group'] == 'Control']
IMS=IAA_full[IAA_full['Group'] == 'Mint']
IH=IAA_full[IAA_full['Group'] == 'Hexanal']
IH.head()

,Group,IAA 0.01,IAA 0.05,IAA 0.1
266,Hexanal,0.210982,0.082528,0.382490
267,Hexanal,0.207433,0.068120,0.402160
268,Hexanal,0.350660,0.130934,0.439151
269,Hexanal,0.099299,0.042892,0.090707
270,Hexanal,0.103156,0.026460,0.154604


In [375]:
# Control, IAA Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Control, IAA Concentration", fontsize=44)
sns.despine(left=True)

#data
d = Ictrl['IAA 0.01']
e = Ictrl['IAA 0.05']
f = Ictrl['IAA 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="IAA 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="IAA 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="IAA 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [376]:
# Hexanal, IAA Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Hexanal, IAA Concentration", fontsize=44)
sns.despine(left=True)

#data
d = IH['IAA 0.01']
e = IH['IAA 0.05']
f = IH['IAA 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="IAA 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="IAA 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="IAA 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [377]:
# Mint, IAA Concentration
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Mint, IAA Concentration", fontsize=44)
sns.despine(left=True)

#data
d = IMS['IAA 0.01']
e = IMS['IAA 0.05']
f = IMS['IAA 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="IAA 0.01")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="IAA 0.05")
sns.distplot(f, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="IAA 0.1")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="b", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="r", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="b",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="r",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

# Stats

Methyl Salicylate

In [8]:
odor = 'MS'

In [9]:
xdf=composite_full[['Group', '%s 0.01' % odor, '%s 0.05' % odor, '%s 0.1' % odor]]

In [11]:
xctrl = xdf[xdf['Group'] == 'Control']
xMS = xdf[xdf['Group'] == 'Mint']
xHex = xdf[xdf['Group'] == 'Hexanal']

In [17]:
kruskal(xctrl['%s 0.01'%odor],xctrl['%s 0.05'%odor],xctrl['%s 0.1'%odor],nan_policy='omit')

KruskalResult(statistic=1.9725528869809088, pvalue=0.37296285696071418)

In [6]:
MSctrl=MS_full[MS_full['Group'] == 'Control']
MSMS=MS_full[MS_full['Group'] == 'Mint']
MShex=MS_full[MS_full['Group'] == 'Hexanal']
MSctrl.tail()

,Group,MS 0.01,MS 0.05,MS 0.1
261,Control,0.297978,0.343609,0.404345
262,Control,0.375490,0.350197,0.420784
263,Control,0.047742,0.213791,0.271523
264,Control,0.159659,0.141683,0.151191
265,Control,0.184315,0.192566,0.260542


In [39]:
kruskal(MSctrl['MS 0.01'],MSctrl['MS 0.05'],MSctrl['MS 0.1'],nan_policy='omit')

KruskalResult(statistic=1.9725528869809088, pvalue=0.37296285696071418)

In [40]:
kruskal(MSMS['MS 0.01'],MSMS['MS 0.05'],MSMS['MS 0.1'],nan_policy='omit')

KruskalResult(statistic=13.539494458225329, pvalue=0.0011479847913647104)

In [41]:
kruskal(MShex['MS 0.01'],MShex['MS 0.05'],MShex['MS 0.1'],nan_policy='omit')

KruskalResult(statistic=5.7933408549275871, pvalue=0.055206729192718254)

Hexanal

In [43]:
Hctrl=H_full[H_full['Group'] == 'Control']
HMS=H_full[H_full['Group'] == 'Mint']
HH=H_full[H_full['Group'] == 'Hexanal']
HH.head()

,Group,Hexanal 0.01,Hexanal 0.05,Hexanal 0.1
266,Hexanal,0.346753,0.300836,0.290083
267,Hexanal,0.269426,0.349465,0.236634
268,Hexanal,0.359969,0.422324,0.416844
269,Hexanal,0.136656,0.135468,0.124364
270,Hexanal,0.188876,0.200218,0.136488


In [44]:
kruskal(Hctrl['Hexanal 0.01'],Hctrl['Hexanal 0.05'],Hctrl['Hexanal 0.1'],nan_policy='omit')

KruskalResult(statistic=4.4640795238193043, pvalue=0.1073093212807168)

In [45]:
kruskal(HMS['Hexanal 0.01'],HMS['Hexanal 0.05'],HMS['Hexanal 0.1'],nan_policy='omit')

KruskalResult(statistic=4.1404582218539545, pvalue=0.12615687447522853)

In [46]:
kruskal(HH['Hexanal 0.01'],HH['Hexanal 0.05'],HH['Hexanal 0.1'],nan_policy='omit')

KruskalResult(statistic=32.384359703646624, pvalue=9.2859350217332081e-08)

IAA

In [47]:
Ictrl=IAA_full[IAA_full['Group'] == 'Control']
IMS=IAA_full[IAA_full['Group'] == 'Mint']
IH=IAA_full[IAA_full['Group'] == 'Hexanal']
IH.head()

,Group,IAA 0.01,IAA 0.05,IAA 0.1
266,Hexanal,0.210982,0.082528,0.382490
267,Hexanal,0.207433,0.068120,0.402160
268,Hexanal,0.350660,0.130934,0.439151
269,Hexanal,0.099299,0.042892,0.090707
270,Hexanal,0.103156,0.026460,0.154604


In [48]:
kruskal(Ictrl['IAA 0.01'],Ictrl['IAA 0.05'],Ictrl['IAA 0.1'],nan_policy='omit')

KruskalResult(statistic=19.499111604184485, pvalue=5.8320563850495121e-05)

In [49]:
kruskal(IMS['IAA 0.01'],IMS['IAA 0.05'],IMS['IAA 0.1'],nan_policy='omit')

KruskalResult(statistic=31.735812204406784, pvalue=1.2842688251749286e-07)

In [50]:
kruskal(IH['IAA 0.01'],IH['IAA 0.05'],IH['IAA 0.1'],nan_policy='omit')

KruskalResult(statistic=38.874224328353328, pvalue=3.6188405076274751e-09)

# By odor

# MS 0.01

In [18]:
MS001_full=composite_full[['Group','MS 0.01']]
MS001df=pd.melt(MS001_full,"Group",var_name="Odor")

In [403]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=MS001df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [404]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=MS001df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [19]:
MS001ctrl=MS001_full[MS001_full['Group'] == 'Control']
MS001MS=MS001_full[MS001_full['Group'] == 'Mint']
MS001H=MS001_full[MS001_full['Group'] == 'Hexanal']
MS001H.head()

,Group,MS 0.01
266,Hexanal,0.161091
267,Hexanal,0.032355
268,Hexanal,0.260874
269,Hexanal,0.034293
270,Hexanal,0.042151


In [406]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("MS 0.01", fontsize=44)
sns.despine(left=True)

#data
d = MS001ctrl['MS 0.01']
e = MS001MS['MS 0.01']
f = MS001H['MS 0.01']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [407]:
kruskal(MS001_full[MS001_full['Group'] == 'Control']['MS 0.01'],MS001_full[MS001_full['Group'] == 'Mint']['MS 0.01'],MS001_full[MS001_full['Group'] == 'Hexanal']['MS 0.01'],nan_policy='omit')

KruskalResult(statistic=17.779511707365906, pvalue=0.00013779329560080323)

# MS 0.05

# ZScore

In [4]:
MS005_full=composite_full[['Group','MS 0.05']]
MS005df=pd.melt(MS005_full,"Group",var_name="Odor")

In [5]:
cg=MS005_full[MS005_full['Group']=='Control']
mg=MS005_full[MS005_full['Group']=='Mint']
hg=MS005_full[MS005_full['Group']=='Hexanal']
hg.head()

,Group,MS 0.05
266,Hexanal,0.096774
267,Hexanal,0.081605
268,Hexanal,0.158723
269,Hexanal,0.024024
270,Hexanal,0.014518


In [6]:
from scipy.stats import zscore

In [66]:
test=zscore(MS005_full[MS005_full['Group']=='Control']['MS 0.05'])
test

array([ -4.72358425e-01,  -4.46490393e-01,  -5.61236468e-01,
        -3.97130501e-01,  -3.96288055e-01,  -5.14181064e-01,
        -3.24306103e-01,  -4.28847851e-01,  -5.59660105e-01,
        -3.40810491e-01,  -4.68637861e-01,  -3.98931745e-01,
        -3.98706274e-01,  -1.09868618e-01,  -1.62971554e-01,
        -3.22466210e-01,  -5.22677717e-01,  -6.53193976e-02,
        -5.30853493e-01,  -4.32004754e-01,  -4.86898362e-01,
        -3.91940616e-01,  -5.57231325e-01,  -5.72096693e-01,
        -2.38257167e-01,   7.04491918e-03,  -5.99119714e-01,
        -5.62094788e-01,  -5.03675898e-01,  -4.56007654e-01,
        -5.58363736e-01,  -4.63235366e-01,  -3.30763631e-01,
        -4.54647986e-01,  -4.34519649e-01,  -5.99119714e-01,
        -2.58499060e-01,  -5.37653922e-01,  -4.70551488e-01,
        -3.84941523e-01,  -5.59134386e-01,  -4.05563636e-01,
        -3.78009481e-01,  -4.40463491e-01,  -4.68281318e-01,
        -5.02582999e-01,  -5.99119714e-01,  -4.48175697e-01,
        -2.02115460e-01,

In [67]:
cgz=zscore(cg['MS 0.05'])
mgz=zscore(mg['MS 0.05'])
hgz=zscore(hg['MS 0.05'])
cgz

array([ -4.72358425e-01,  -4.46490393e-01,  -5.61236468e-01,
        -3.97130501e-01,  -3.96288055e-01,  -5.14181064e-01,
        -3.24306103e-01,  -4.28847851e-01,  -5.59660105e-01,
        -3.40810491e-01,  -4.68637861e-01,  -3.98931745e-01,
        -3.98706274e-01,  -1.09868618e-01,  -1.62971554e-01,
        -3.22466210e-01,  -5.22677717e-01,  -6.53193976e-02,
        -5.30853493e-01,  -4.32004754e-01,  -4.86898362e-01,
        -3.91940616e-01,  -5.57231325e-01,  -5.72096693e-01,
        -2.38257167e-01,   7.04491918e-03,  -5.99119714e-01,
        -5.62094788e-01,  -5.03675898e-01,  -4.56007654e-01,
        -5.58363736e-01,  -4.63235366e-01,  -3.30763631e-01,
        -4.54647986e-01,  -4.34519649e-01,  -5.99119714e-01,
        -2.58499060e-01,  -5.37653922e-01,  -4.70551488e-01,
        -3.84941523e-01,  -5.59134386e-01,  -4.05563636e-01,
        -3.78009481e-01,  -4.40463491e-01,  -4.68281318e-01,
        -5.02582999e-01,  -5.99119714e-01,  -4.48175697e-01,
        -2.02115460e-01,

In [63]:
sns.distplot(cgz)
sns.distplot(mgz)
sns.distplot(hgz)

In [75]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.distplot(cgz, color="r", kde_kws={"shade": True},hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,label="Control")
sns.distplot(mgz, color="g", kde_kws={"shade": True},hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,label="Mint")
sns.distplot(hgz, color="b", kde_kws={"shade": True},hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,label="Hexanal")
sns.despine();
plt.legend(loc='upper right');
plt.xlabel('Z-score');
plt.title('Z-scores for MS');

# Normalize by kde peak

In [5]:
MS005_full=composite_full[['Group','MS 0.05']]
MS005df=pd.melt(MS005_full,"Group",var_name="Odor")
cg=MS005_full[MS005_full['Group']=='Control']
mg=MS005_full[MS005_full['Group']=='Mint']
hg=MS005_full[MS005_full['Group']=='Hexanal']

In [6]:
from scipy.stats import gaussian_kde

In [7]:
from scipy.signal import find_peaks_cwt

In [8]:
gkde=gaussian_kde(cg['MS 0.05'])

In [10]:
sns.distplot(cg['MS 0.05'])

In [15]:
x=np.arange(-1,5,0.01)
y=gkde.evaluate(y)

In [22]:
plt.plot(x,y)

In [23]:
from scipy import signal

In [34]:
np.amax(y)

1.1579025390002664

In [410]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=MS005df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [411]:
MS005ctrl=MS005_full[MS005_full['Group'] == 'Control']
MS005MS=MS005_full[MS005_full['Group'] == 'Mint']
MS005H=MS005_full[MS005_full['Group'] == 'Hexanal']
MS005H.head()

,Group,MS 0.05
266,Hexanal,0.096774
267,Hexanal,0.081605
268,Hexanal,0.158723
269,Hexanal,0.024024
270,Hexanal,0.014518


In [412]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("MS 0.05", fontsize=44)
sns.despine(left=True)

#data
d = MS005ctrl['MS 0.05']
e = MS005MS['MS 0.05']
f = MS005H['MS 0.05']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [413]:
kruskal(MS005_full[MS005_full['Group'] == 'Control']['MS 0.05'],MS005_full[MS005_full['Group'] == 'Mint']['MS 0.05'],MS005_full[MS005_full['Group'] == 'Hexanal']['MS 0.05'],nan_policy='omit')

KruskalResult(statistic=22.348493455193285, pvalue=1.4030925116132286e-05)

# MS 0.1

In [414]:
MS10_full=composite_full[['Group','MS 0.1']]
MS10df=pd.melt(MS10_full,"Group",var_name="Odor")

In [415]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=MS10df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [416]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=MS10df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [417]:
MS10ctrl=MS10_full[MS10_full['Group'] == 'Control']
MS10MS=MS10_full[MS10_full['Group'] == 'Mint']
MS10H=MS10_full[MS10_full['Group'] == 'Hexanal']
MS10H.head()

,Group,MS 0.1
266,Hexanal,0.149891
267,Hexanal,0.094942
268,Hexanal,0.250858
269,Hexanal,0.048251
270,Hexanal,0.080076


In [418]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("MS 0.1", fontsize=44)
sns.despine(left=True)

#data
d = MS10ctrl['MS 0.1']
e = MS10MS['MS 0.1']
f = MS10H['MS 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [126]:
kruskal(THA_full[THA_full['Group'] == 'Control']['THA'],THA_full[THA_full['Group'] == 'Mint']['THA'],THA_full[THA_full['Group'] == 'Hexanal']['THA'],nan_policy='omit')

KruskalResult(statistic=26.646950444965789, pvalue=1.6356419143785338e-06)

# IAA 1%

In [419]:
IAA001_full=composite_full[['Group','IAA 0.01']]
IAA001df=pd.melt(IAA001_full,"Group",var_name="Odor")

In [420]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAA001df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [421]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAA001df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [422]:
IAA001ctrl=IAA001_full[IAA001_full['Group'] == 'Control']
IAA001MS=IAA001_full[IAA001_full['Group'] == 'Mint']
IAA001H=IAA001_full[IAA001_full['Group'] == 'Hexanal']
IAA001H.head()

,Group,IAA 0.01
266,Hexanal,0.210982
267,Hexanal,0.207433
268,Hexanal,0.350660
269,Hexanal,0.099299
270,Hexanal,0.103156


In [423]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("IAA 0.01", fontsize=44)
sns.despine(left=True)

#data
d = IAA001ctrl['IAA 0.01']
e = IAA001MS['IAA 0.01']
f = IAA001H['IAA 0.01']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [424]:
kruskal(IAA001_full[IAA001_full['Group'] == 'Control']['IAA 0.01'],IAA001_full[IAA001_full['Group'] == 'Mint']['IAA 0.01'],IAA001_full[IAA001_full['Group'] == 'Hexanal']['IAA 0.01'],nan_policy='omit')

KruskalResult(statistic=22.979999688546172, pvalue=1.0231904325257964e-05)

# IAA 0.05

In [425]:
IAA005_full=composite_full[['Group','IAA 0.05']]
IAA005df=pd.melt(IAA005_full,"Group",var_name="Odor")

In [426]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAA005df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [427]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAA005df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [428]:
IAA005ctrl=IAA005_full[IAA005_full['Group'] == 'Control']
IAA005MS=IAA005_full[IAA005_full['Group'] == 'Mint']
IAA005H=IAA005_full[IAA005_full['Group'] == 'Hexanal']
IAA005H.head()

,Group,IAA 0.05
266,Hexanal,0.082528
267,Hexanal,0.068120
268,Hexanal,0.130934
269,Hexanal,0.042892
270,Hexanal,0.026460


In [429]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("IAA 0.05", fontsize=44)
sns.despine(left=True)

#data
d = IAA005ctrl['IAA 0.05']
e = IAA005MS['IAA 0.05']
f = IAA005H['IAA 0.05']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [430]:
kruskal(IAA005_full[IAA005_full['Group'] == 'Control']['IAA 0.05'],IAA005_full[IAA005_full['Group'] == 'Mint']['IAA 0.05'],IAA005_full[IAA005_full['Group'] == 'Hexanal']['IAA 0.05'],nan_policy='omit')

KruskalResult(statistic=7.2116318108394726, pvalue=0.027165271476384416)

# IAA 0.1

In [437]:
IAA10_full=composite_full[['Group','IAA 0.1']]
IAA10df=pd.melt(IAA10_full,"Group",var_name="Odor")

In [438]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAA10df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [439]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=IAA10df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [440]:
IAA10ctrl=IAA10_full[IAA10_full['Group'] == 'Control']
IAA10MS=IAA10_full[IAA10_full['Group'] == 'Mint']
IAA10H=IAA10_full[IAA10_full['Group'] == 'Hexanal']
IAA10H.head()

,Group,IAA 0.1
266,Hexanal,0.382490
267,Hexanal,0.402160
268,Hexanal,0.439151
269,Hexanal,0.090707
270,Hexanal,0.154604


In [441]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("IAA 0.1", fontsize=44)
sns.despine(left=True)

#data
d = IAA10ctrl['IAA 0.1']
e = IAA10MS['IAA 0.1']
f = IAA10H['IAA 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [436]:
kruskal(IAA10_full[IAA10_full['Group'] == 'Control']['IAA 0.1'],IAA10_full[IAA10_full['Group'] == 'Mint']['IAA 0.1'],IAA10_full[IAA10_full['Group'] == 'Hexanal']['IAA 0.1'],nan_policy='omit')

KruskalResult(statistic=30.073164294270121, pvalue=2.9491397092061267e-07)

# Hexanal 1%

In [448]:
H001_full=composite_full[['Group','Hexanal 0.01']]
H001df=pd.melt(H001_full,"Group",var_name="Odor")

In [449]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=H001df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [450]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=H001df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [451]:
H001ctrl=H001_full[H001_full['Group'] == 'Control']
H001MS=H001_full[H001_full['Group'] == 'Mint']
H001H=H001_full[H001_full['Group'] == 'Hexanal']
H001H.head()

,Group,Hexanal 0.01
266,Hexanal,0.346753
267,Hexanal,0.269426
268,Hexanal,0.359969
269,Hexanal,0.136656
270,Hexanal,0.188876


In [452]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Hexanal 0.01", fontsize=44)
sns.despine(left=True)

#data
d = H001ctrl['Hexanal 0.01']
e = H001MS['Hexanal 0.01']
f = H001H['Hexanal 0.01']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [453]:
kruskal(H001_full[H001_full['Group'] == 'Control']['Hexanal 0.01'],H001_full[H001_full['Group'] == 'Mint']['Hexanal 0.01'],H001_full[H001_full['Group'] == 'Hexanal']['Hexanal 0.01'],nan_policy='omit')

KruskalResult(statistic=24.469118887167379, pvalue=4.8595756261645399e-06)

# Hexanal 0.05

In [455]:
H005_full=composite_full[['Group','Hexanal 0.05']]
H005df=pd.melt(H005_full,"Group",var_name="Odor")

In [456]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=H005df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [457]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=H005df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [458]:
H005ctrl=H005_full[H005_full['Group'] == 'Control']
H005MS=H005_full[H005_full['Group'] == 'Mint']
H005H=H005_full[H005_full['Group'] == 'Hexanal']
H005H.head()

,Group,Hexanal 0.05
266,Hexanal,0.300836
267,Hexanal,0.349465
268,Hexanal,0.422324
269,Hexanal,0.135468
270,Hexanal,0.200218


In [460]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Hexanal 0.05", fontsize=44)
sns.despine(left=True)

#data
d = H005ctrl['Hexanal 0.05']
e = H005MS['Hexanal 0.05']
f = H005H['Hexanal 0.05']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [461]:
kruskal(H005_full[H005_full['Group'] == 'Control']['Hexanal 0.05'],H005_full[H005_full['Group'] == 'Mint']['Hexanal 0.05'],H005_full[H005_full['Group'] == 'Hexanal']['Hexanal 0.05'],nan_policy='omit')

KruskalResult(statistic=66.921585593621458, pvalue=2.9387478759540041e-15)

# Hexanal 0.1

In [462]:
H10_full=composite_full[['Group','Hexanal 0.1']]
H10df=pd.melt(H10_full,"Group",var_name="Odor")

In [463]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=H10df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [464]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=H10df);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [465]:
H10ctrl=H10_full[H10_full['Group'] == 'Control']
H10MS=H10_full[H10_full['Group'] == 'Mint']
H10H=H10_full[H10_full['Group'] == 'Hexanal']
H10H.head()

,Group,Hexanal 0.1
266,Hexanal,0.290083
267,Hexanal,0.236634
268,Hexanal,0.416844
269,Hexanal,0.124364
270,Hexanal,0.136488


In [466]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("IAA 0.05", fontsize=44)
sns.despine(left=True)

#data
d = H10ctrl['Hexanal 0.1']
e = H10MS['Hexanal 0.1']
f = H10H['Hexanal 0.1']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [468]:
kruskal(H10_full[H10_full['Group'] == 'Control']['Hexanal 0.1'],H10_full[dH10_full['Group'] == 'Mint']['Hexanal 0.1'],H10_full[H10_full['Group'] == 'Hexanal']['Hexanal 0.1'],nan_policy='omit')

KruskalResult(statistic=15.839109543591839, pvalue=0.00036356415948386392)

# THA

In [296]:
THA_full=composite_full[['Group','THA']]
THAdf=pd.melt(THA_full,"Group",var_name="Odor")

In [360]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=THAdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');

In [362]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=THAdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [341]:
THActrl=THA_full[THA_full['Group'] == 'Control']
THAMS=THA_full[THA_full['Group'] == 'Mint']
THAH=THA_full[THA_full['Group'] == 'Hexanal']
THAH.head()

,Group,THA
266,Hexanal,0.032360
267,Hexanal,0.068896
268,Hexanal,0.141124
269,Hexanal,0.031333
270,Hexanal,0.029448


In [357]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("2-hydroxyacetophenone", fontsize=44)
sns.despine(left=True)

#data
d = THActrl['THA']
e = THAMS['THA']
f = THAH['THA']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [126]:
kruskal(THA_full[THA_full['Group'] == 'Control']['THA'],THA_full[THA_full['Group'] == 'Mint']['THA'],THA_full[THA_full['Group'] == 'Hexanal']['THA'],nan_policy='omit')

KruskalResult(statistic=26.646950444965789, pvalue=1.6356419143785338e-06)

# AP

In [264]:
AP_full=composite_full[['Group','AP']]
APdf=pd.melt(AP_full,"Group",var_name="Odor")

In [335]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=APdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [336]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=APdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [358]:
APctrl=AP_full[AP_full['Group'] == 'Control']
APMS=AP_full[AP_full['Group'] == 'Mint']
APH=AP_full[AP_full['Group'] == 'Hexanal']
APH.head()

,Group,AP
266,Hexanal,0.150098
267,Hexanal,0.074321
268,Hexanal,0.180250
269,Hexanal,0.113269
270,Hexanal,0.028062


In [359]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("2-hydroxyacetophenone", fontsize=44)
sns.despine(left=True)

#data
d = APctrl['AP']
e = APMS['AP']
f = APH['AP']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [131]:
kruskal(AP_full[AP_full['Group'] == 'Control']['AP'],AP_full[AP_full['Group'] == 'Mint']['AP'],AP_full[AP_full['Group'] == 'Hexanal']['AP'],nan_policy='omit')

KruskalResult(statistic=46.307698447912315, pvalue=8.7985488016197506e-11)

# EB

In [332]:
EB_full=composite_full[['Group','EB']]
EBdf=pd.melt(EB_full,"Group",var_name="Odor")

In [337]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=EBdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [338]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=EBdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [363]:
EBctrl=EB_full[EB_full['Group'] == 'Control']
EBMS=EB_full[EB_full['Group'] == 'Mint']
EBH=EB_full[EB_full['Group'] == 'Hexanal']
EBH.head()

,Group,EB
266,Hexanal,0.209884
267,Hexanal,0.193009
268,Hexanal,0.335625
269,Hexanal,0.082340
270,Hexanal,0.023194


In [364]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("2-hydroxyacetophenone", fontsize=44)
sns.despine(left=True)

#data
d = EBctrl['EB']
e = EBMS['EB']
f = EBH['EB']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [135]:
kruskal(EB_full[EB_full['Group'] == 'Control']['EB'],EB_full[EB_full['Group'] == 'Mint']['EB'],EB_full[EB_full['Group'] == 'Hexanal']['EB'],nan_policy='omit')

KruskalResult(statistic=20.686023094405726, pvalue=3.2217152941460278e-05)

# Hexanone

In [378]:
Hone_full=composite_full[['Group','Hexanone']]
Honedf=pd.melt(Hone_full,"Group",var_name="Odor")

In [379]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=Honedf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [380]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=Honedf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [381]:
Honectrl=Hone_full[Hone_full['Group'] == 'Control']
HoneMS=Hone_full[Hone_full['Group'] == 'Mint']
HoneH=Hone_full[Hone_full['Group'] == 'Hexanal']
HoneH.head()

,Group,Hexanone
266,Hexanal,0.378703
267,Hexanal,0.330326
268,Hexanal,0.501147
269,Hexanal,0.073911
270,Hexanal,0.117332


In [382]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("Hexanone", fontsize=44)
sns.despine(left=True)

#data
d = Honectrl['Hexanone']
e = HoneMS['Hexanone']
f = HoneH['Hexanone']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [383]:
kruskal(Hone_full[Hone_full['Group'] == 'Control']['Hexanone'],Hone_full[Hone_full['Group'] == 'Mint']['Hexanone'],Hone_full[Hone_full['Group'] == 'Hexanal']['Hexanone'],nan_policy='omit')

KruskalResult(statistic=20.558063665415894, pvalue=3.4345765132367753e-05)

# PA

In [384]:
PA_full=composite_full[['Group','PA']]
PAdf=pd.melt(PA_full,"Group",var_name="Odor")

In [385]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=PAdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [386]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=PAdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [387]:
PActrl=PA_full[PA_full['Group'] == 'Control']
PAMS=PA_full[PA_full['Group'] == 'Mint']
PAH=PA_full[PA_full['Group'] == 'Hexanal']
PAH.head()

,Group,PA
266,Hexanal,0.043617
267,Hexanal,0.032337
268,Hexanal,0.000000
269,Hexanal,0.004714
270,Hexanal,0.032430


In [388]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("2-hydroxyacetophenone", fontsize=44)
sns.despine(left=True)

#data
d = PActrl['PA']
e = PAMS['PA']
f = PAH['PA']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [389]:
kruskal(PA_full[PA_full['Group'] == 'Control']['PA'],PA_full[PA_full['Group'] == 'Mint']['PA'],PA_full[PA_full['Group'] == 'Hexanal']['PA'],nan_policy='omit')

KruskalResult(statistic=36.777881600341829, pvalue=1.032249027618262e-08)

# Blank

In [391]:
B_full=composite_full[['Group','BLANK']]
Bdf=pd.melt(B_full,"Group",var_name="Odor")

In [392]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=Bdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [393]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=3);
plt.figure(figsize=(15,18));
sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=Bdf);
sns.despine();
plt.legend(loc='upper right');
plt.ylabel('Peak DF/F');
plt.title('Peak DF/F');
plt.xlabel('Odor');

In [398]:
Bctrl=B_full[B_full['Group'] == 'Control']
Bctrl=Bctrl.dropna()
BMS=B_full[B_full['Group'] == 'Mint']
BMS=BMS.dropna()
BH=B_full[B_full['Group'] == 'Hexanal']
BH=BH.dropna()
BH.head()

,Group,BLANK
287,Hexanal,0.059379
288,Hexanal,0.027291
289,Hexanal,0.069221
290,Hexanal,0.121546
291,Hexanal,0.055183


In [399]:
sns.set(style="white", palette="muted", color_codes=True)
sns.set_context("talk", font_scale=2)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(30,20), sharex=True)
# f.suptitle("2-hydroxyacetophenone", fontsize=44)
sns.despine(left=True)

#data
d = Bctrl['BLANK']
e = BMS['BLANK']
f = BH['BLANK']

# Plot a simple histogram with binsize determined automatically
sns.distplot(d, kde=False, color="r", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(e, kde=False, color="g", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])
sns.distplot(f, kde=False, color="b", hist_kws={"histtype":'step',"linewidth":3,"alpha":0.7},axlabel= False,ax=axes[0, 0])

# Plot a kernel density estimate and rug plot
sns.distplot(d, hist=False, rug=True, color="r", axlabel= False,ax=axes[0, 1],label="Control")
sns.distplot(e, hist=False, rug=True, color="g", axlabel= False,ax=axes[0, 1],label="Mint")
sns.distplot(f, hist=False, rug=True, color="b", axlabel= False,ax=axes[0, 1],label="Hexanal")

# Plot a filled kernel density estimate
sns.distplot(d, hist=False, color="r", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(e, hist=False, color="g", kde_kws={"shade": True}, axlabel=False,ax=axes[1, 0])
sns.distplot(f, hist=False, color="b", kde_kws={"shade": True}, axlabel= False,ax=axes[1, 0])

# Plot a historgram and kernel density estimate
sns.distplot(d, color="r",axlabel= False,ax=axes[1, 1])
sns.distplot(e, color="g",axlabel= False,ax=axes[1, 1])
sns.distplot(f, color="b",axlabel= False,ax=axes[1, 1])

plt.setp(axes, yticks=[])
plt.tight_layout()

In [400]:
kruskal(B_full[B_full['Group'] == 'Control']['BLANK'],B_full[B_full['Group'] == 'Mint']['BLANK'],B_full[B_full['Group'] == 'Hexanal']['BLANK'],nan_policy='omit')

KruskalResult(statistic=109.93241092905365, pvalue=1.3442507132896537e-24)

# Baseline, by group

In [23]:
bl=pd.read_csv('C:\Users\Annie\Documents\Data\Ca_Imaging\Analysis\Odor_Panel\Composite_Baseline_NoP.csv')
del bl['Mouse']
bl_sorted=bl.reindex_axis(bl.mean().sort_values().index, axis=1)
bl_labels=pd.DataFrame(bl.Group)
tmp=[bl_labels,bl_sorted]
bdf=pd.concat(tmp,axis=1)
bdfull=pd.melt(bdf,"Group",var_name="Odor")
bdfull=bdfull.dropna()

# Composite Graphs - Baseline

In [498]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=2.2);
plt.figure(figsize=(45,20));
ax=sns.barplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=bdfull);
sns.despine()
plt.ylabel('Intensity', fontsize=48);
plt.title('Baseline F',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':30});

In [518]:
# sns.set_context("talk",font_scale=1.9);
# sns.set(style="white", palette="muted", color_codes=True);
# plt.figure(figsize=(8, 6));
# violinplot=sns.violinplot(x="Odor", y="value", hue="Group", data=bdfull);
# violinplot.set(ylabel='Intensity');

In [502]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=2.2);
plt.figure(figsize=(45,20));
ax=sns.boxplot(x="Odor", y="value", hue="Group",palette={"Control": "r", "Hexanal": "b","Mint":"g"}, data=bdfull);
sns.despine()
plt.ylabel('Intensity', fontsize=48);
plt.title('Baseline F',fontsize=55);
plt.xlabel('Odor',fontsize=48);
plt.legend(loc=2,prop={'size':30});

# Odor, by Mouse

In [20]:
readms=pd.read_csv('C:\Users\Annie\Documents\Data\Ca_Imaging\Analysis\Odor_Panel\Composite_MaxDF_NoP.csv')
ms = readms
del ms['Group']
ms_sorted=ms.reindex_axis(ms.mean().sort_values().index, axis=1)
ms_labels=pd.DataFrame(ms.Mouse)
tmp=[ms_labels,ms_sorted]
msdf=pd.concat(tmp,axis=1)
msfull=pd.melt(msdf,"Mouse",var_name="Odor")
msfull=msfull.dropna()
msfull.head()

,Mouse,Odor,value
0,160321_3,THA,0.212200
1,160321_3,THA,0.066122
2,160321_3,THA,0.022259
3,160321_3,THA,0.145203
4,160321_3,THA,0.155862


In [522]:
#dictionary of imaging session and labels
msgroups={'160321_3':'r', '160421_2':'r', '160502_1':'r', '160503_1':'r', '160420_1':'r',
       '160420_2':'r', '160420_3':'r', '160421_1':'r', '160421_3':'r', '160503_2':'r',
       '160310_2':'r', '160310_3':'r', '160321_1':'r', '160321_2':'r', '160517_2':'b',
       '160525_1':'b', '160620_1':'b', '160517_1':'b', '160517_3':'b', '160525_2':'b',
       '160525_3':'b', '160620_2':'b', '160620_3':'b', '160621_1':'b', '160621_2':'b',
       '160622_1':'b', '160622_2':'b', '160626_1':'b', '160626_2':'b', '160330_3':'g',
       '160328_1_B':'g', '160428_1_B':'g', '160429_1':'g', '160429_2':'g', '160325_1':'g',
       '160325_2':'g', '160325_4':'g', '160328_2':'g', '160328_3':'g', '160330_2':'g',
       '160401_1':'g'}

# Composite Graphs - by session

In [529]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=2.2);
plt.figure(figsize=(45,20));
ax=sns.barplot(x="Odor", y="value", hue="Mouse",palette={'160321_3':'r', '160421_2':'r', '160502_1':'r', '160503_1':'r', '160420_1':'r',
       '160420_2':'r', '160420_3':'r', '160421_1':'r', '160421_3':'r', '160503_2':'r',
       '160310_2':'r', '160310_3':'r', '160321_1':'r', '160321_2':'r', '160517_2':'b',
       '160525_1':'b', '160620_1':'b', '160517_1':'b', '160517_3':'b', '160525_2':'b',
       '160525_3':'b', '160620_2':'b', '160620_3':'b', '160621_1':'b', '160621_2':'b',
       '160622_1':'b', '160622_2':'b', '160626_1':'b', '160626_2':'b', '160330_3':'g',
       '160328_1_B':'g', '160428_1_B':'g', '160429_1':'g', '160429_2':'g', '160325_1':'g',
       '160325_2':'g', '160325_4':'g', '160328_2':'g', '160328_3':'g', '160330_2':'g',
       '160401_1':'g'}, data=msfull);
sns.despine()
plt.ylabel('Peak DF/F', fontsize=48);
plt.title('Odor-evoked response, by session',fontsize=55);
plt.xlabel('Odor',fontsize=48);
ax.legend_.remove()

In [14]:
# sns.set(style="white", palette="muted", color_codes=True);
# sns.set_context("talk",font_scale=2.2);
# plt.figure(figsize=(45,20));
# ax=sns.violinplot(x="Odor", y="value", hue="Mouse",palette={'160321_3':'r', '160421_2':'r', '160502_1':'r', '160503_1':'r', '160420_1':'r',
#        '160420_2':'r', '160420_3':'r', '160421_1':'r', '160421_3':'r', '160503_2':'r',
#        '160310_2':'r', '160310_3':'r', '160321_1':'r', '160321_2':'r', '160517_2':'b',
#        '160525_1':'b', '160620_1':'b', '160517_1':'b', '160517_3':'b', '160525_2':'b',
#        '160525_3':'b', '160620_2':'b', '160620_3':'b', '160621_1':'b', '160621_2':'b',
#        '160622_1':'b', '160622_2':'b', '160626_1':'b', '160626_2':'b', '160330_3':'g',
#        '160328_1_B':'g', '160428_1_B':'g', '160429_1':'g', '160429_2':'g', '160325_1':'g',
#        '160325_2':'g', '160325_4':'g', '160328_2':'g', '160328_3':'g', '160330_2':'g',
#        '160401_1':'g'}, data=msfull);
# sns.despine()
# plt.ylabel('Peak DF/F', fontsize=48);
# plt.title('Odor-evoked response, by session',fontsize=55);
# plt.xlabel('Odor',fontsize=48);
# ax.legend_.remove()

In [531]:
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk",font_scale=2.2);
plt.figure(figsize=(55,40));
ax=sns.boxplot(x="Odor", y="value", hue="Mouse",palette={'160321_3':'r', '160421_2':'r', '160502_1':'r', '160503_1':'r', '160420_1':'r',
       '160420_2':'r', '160420_3':'r', '160421_1':'r', '160421_3':'r', '160503_2':'r',
       '160310_2':'r', '160310_3':'r', '160321_1':'r', '160321_2':'r', '160517_2':'b',
       '160525_1':'b', '160620_1':'b', '160517_1':'b', '160517_3':'b', '160525_2':'b',
       '160525_3':'b', '160620_2':'b', '160620_3':'b', '160621_1':'b', '160621_2':'b',
       '160622_1':'b', '160622_2':'b', '160626_1':'b', '160626_2':'b', '160330_3':'g',
       '160328_1_B':'g', '160428_1_B':'g', '160429_1':'g', '160429_2':'g', '160325_1':'g',
       '160325_2':'g', '160325_4':'g', '160328_2':'g', '160328_3':'g', '160330_2':'g',
       '160401_1':'g'}, data=msfull);
sns.despine()
plt.ylabel('Peak DF/F', fontsize=48);
plt.title('Odor-evoked response, by session',fontsize=55);
plt.xlabel('Odor',fontsize=48);
ax.legend_.remove()